In [9]:
GYRO = False
SEQUENCE_LENGTH = 4
SEQUENCE_OVERLAP = 3
BATCH_SIZE = 10
EPOCHS = 100
MODEL_NAME = f"position_epochs:{EPOCHS}_batch:{BATCH_SIZE}_gyro:{GYRO}_window:{SEQUENCE_LENGTH}_overlap:{SEQUENCE_OVERLAP}"

### This code trains a classifier that returns what position the activity is in, eg "sitting_or_standing", "lying_down_left" etc


In [10]:
import file_tagger
import sequence_genrator
import tensorflow as tf
import numpy as np
from keras import layers, Sequential, models
from sklearn.model_selection import train_test_split

POSITIONS = {
    "sitting_or_standing",
    "lying_down_back",
    "lying_down_stomach",
    "lying_down_right",
    "lying_down_left"
}


TASK3_ACTIVITIES = {
    "sitting&coughing",
    "sitting&hyperventilating",
    "sitting&normal_breathing",
    
    "standing&coughing",
    "standing&hyperventilating",
    "standing&normal_breathing",
    
    "lying_down_back&coughing",
    "lying_down_back&hyperventilating",
    "lying_down_back&normal_breathing",
    
    "lying_down_stomach&coughing",
    "lying_down_stomach&hyperventilating",
    "lying_down_stomach&normal_breathing",
    
    "lying_down_right&coughing",
    "lying_down_right&hyperventilating",
    "lying_down_right&normal_breathing",
    
    "lying_down_left&coughing",
    "lying_down_left&hyperventilating",
    "lying_down_left&normal_breathing",
    
    "sitting&talking",
    "sitting&eating",
    "sitting&singing",
    "sitting&laughing",
    
    "standing&talking",
    "standing&eating",
    "standing&singing",
    "standing&laughing",
    
    "lying_down_back&talking",
    "lying_down_back&singing",
    "lying_down_back&laughing",
    
    "lying_down_right&talking",
    "lying_down_right&singing",
    "lying_down_right&laughing",
    
    "lying_down_left&talking",
    "lying_down_left&singing",
    "lying_down_left&laughing",
    
    "lying_down_stomach&talking",
    "lying_down_stomach&singing",
    "lying_down_stomach&laughing",
}

DATA_DIRECTORY = "./all_respeck"

In [11]:
# split data into training, dev, and test sets
def train_dev_test_split(data, labels, dev_size, test_size):
    # Split the data into training and temporary (dev + test) sets
    train_data, temp_data, train_labels, temp_labels = train_test_split(data, labels, test_size=(dev_size + test_size))
    
    # Split the temporary data into dev and test sets
    dev_data, test_data, dev_labels, test_labels = train_test_split(temp_data, temp_labels, 
                                                                 test_size=(test_size / (dev_size + test_size)))
    
    return train_data, dev_data, test_data, train_labels, dev_labels, test_labels

In [12]:
# generate training data
def generate_training_data(directory, sequence_length, overlap, gyro = GYRO): # if gyro is false, only accelerometer data is used

    tagged_data = []

    # group each csv file into their respective areas
    csv_dictionary = file_tagger.tag_directory(directory)

    # iterates through each activity
    for key in TASK3_ACTIVITIES:

        # iterates through each csv file for the activity 
        for csv_file in csv_dictionary[key]:
            if gyro:
                sequences = sequence_genrator.generate_sequences_from_file_with_gyroscope(directory + "/" + csv_file, sequence_length, overlap)
            else:
                sequences = sequence_genrator.generate_sequences_from_file_without_gyroscope(directory + "/" + csv_file, sequence_length, overlap)

            # iterate through each generated sequence
            for sequence in sequences:
                tagged_data.append((key, sequence))

    print ("there are " + str(len(tagged_data)) + " tagged sequences in the dataset")
    return tagged_data

In [13]:
#CNN MODEL
def train_model_CNN(input_data, labels_encoded, unique_labels, epochs, batch_size, validation_data):
    if GYRO:
        width = 6
    else:
        width = 3
    # Define the CNN model for your specific input shape
    model = Sequential([
        layers.Conv1D(32, 3, activation='relu', input_shape=(SEQUENCE_LENGTH*25, width)),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(128, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Dropout(0.5),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(len(unique_labels), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    return model

In [14]:

def tag_data_with_position(tagged_data):
    tagged_data_with_position = []
    for tag, sequence in tagged_data:
        position = tag.split("&")[0]
        if position == "sitting" or position == "standing":
            position = "sitting_or_standing"
        tagged_data_with_position.append((position, sequence))
    return tagged_data_with_position

def generate_position_model(tagged_sequences):
    tagged_sequences_with_position = tag_data_with_position(tagged_sequences)
    # Combine all sequences and labels
    sequences = [sequence for _, sequence in tagged_sequences_with_position]
    labels = [label for label, _ in tagged_sequences_with_position]


    # encode labels to numbers
    sequences = np.array(sequences, dtype=np.float32)
    label_to_index = {label: idx for idx, label in enumerate(POSITIONS)}
    labels_encoded = [label_to_index[label] for label in labels]
    labels_encoded = np.array(labels_encoded)

    train_data, dev_data, test_data, train_labels, dev_labels, test_labels = train_dev_test_split(sequences, labels_encoded, dev_size=0.1, test_size=0.1) #10% dev, 10% test


    # train and save model (CHOOSE BETWEEN CNN AND LSTM)
    model = train_model_CNN(train_data, train_labels, POSITIONS, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(dev_data, dev_labels)) #batch_size, epochs


    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(test_data, test_labels)
    print (f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")
    return model

In [15]:
tagged_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, GYRO)
model = generate_position_model(tagged_sequences)

# Save the trained model
#model.save(f"models/Task3/{MODEL_NAME}_{test_accuracy}.keras")

there are 47124 tagged sequences in the dataset
Epoch 1/100
3770/3770 [==============================] - 12s 3ms/step - loss: 0.1258 - accuracy: 0.9699 - val_loss: 0.0742 - val_accuracy: 0.9801
Epoch 2/100
3770/3770 [==============================] - 11s 3ms/step - loss: 0.0887 - accuracy: 0.9763 - val_loss: 0.0692 - val_accuracy: 0.9790
Epoch 3/100
3770/3770 [==============================] - 11s 3ms/step - loss: 0.0765 - accuracy: 0.9778 - val_loss: 0.0558 - val_accuracy: 0.9826
Epoch 4/100
3770/3770 [==============================] - 11s 3ms/step - loss: 0.0706 - accuracy: 0.9798 - val_loss: 0.0621 - val_accuracy: 0.9843
Epoch 5/100
3770/3770 [==============================] - 11s 3ms/step - loss: 0.0669 - accuracy: 0.9801 - val_loss: 0.0629 - val_accuracy: 0.9830
Epoch 6/100
3770/3770 [==============================] - 11s 3ms/step - loss: 0.0678 - accuracy: 0.9803 - val_loss: 0.0745 - val_accuracy: 0.9811
Epoch 7/100
3770/3770 [==============================] - 11s 3ms/step - loss

KeyboardInterrupt: 